In [ ]:
import _09_utility as util

dataset = util.load_large_csv('spx_features.csv')
fssr = util.fn_all_subsets(True,0,True)
fssd = util.fnsubset_to_indexdictlist(dataset.columns,fssr)
fss = []
for d in fssd:
    fss.append(list(d.keys()))
print(fss)
dataset = dataset.values
arr_low = dataset[:,1]
arr_close = dataset[:,2]

In [ ]:
#example for initiation and evaluation steps

from importlib import reload
import _00_gene as _0
reload(_0)
import _01_initialization as _1
reload(_1)
import _02_evaluation as _2
reload(_2)
import _03_elitism as _3
reload(_3)
import _04_parentselection as _4
reload(_4)
import _05_reproduction as _5
reload(_5)
import _06_mutation as _6
import _09_utility as util


#dataset = util.load_large_csv('spx_features.csv')
#dataset, fss = util.drop_all_targets()

log_normalize = True
lag_allowance = 30
generations = 10
hold_for = 15

#kelsch_ratio
#average_return
#profit_factor

criteria = 'kelsch_ratio'

g1 = _0.Gene()

g1.custom(
	fss=fssd,
	acceptable_lag=lag_allowance,
	pattern_vals=[
		(412, 10, 'gt', 412, 14),
		(454, 8, 'gt', 464, 24)
	]
)

new_population = _5.evolutionary_branch(
	gene=g1,
	branch_size=100
)

print("collecting returns and kratio... ",end='')
arr_returns, arr_kratio = _1.collect_parallel_metrics(
	arr_close=arr_close,
	arr_low=arr_low,
	hold_for=hold_for,
	lag_allow=lag_allowance,
	log_normalize=log_normalize
)
print("complete.")

#begin generational loop
for generation in range(generations):

	population = new_population

	print(f"Generation {generation+1} ({criteria}): ",end='')

	returns, kelsch_ratio = _2.fitness(
		arr_close=arr_close,
		arr_low=arr_low,
		arr_returns=arr_returns,
		arr_kratio=arr_kratio,
		data=dataset,
		genes= population,
		hold_for=hold_for,
		lag_allow=lag_allowance,
		specific_data=None,#'form_519',
		log_normalize=log_normalize
	)
	unsorted_population = _2.associate(
		genes=population,
		returns=returns,
		kelsch_ratio=kelsch_ratio,
		log_normalize=log_normalize
	)

	unsorted_population = _2.filter_population(
		population=unsorted_population,
		avg_return=2.00 / 25000,
		tot_return=0.25,
		profit_factor=0,
		kelsch_ratio=2.0,
		entry_frequency=0.00
	)

	population = _2.sort_population(
		population=unsorted_population,
		criteria=criteria
	)

	avg, top = _2.simple_generational_stat_output(population,criteria)
	print(f"AVG {avg}, BEST {top}, FROM {len(population)} GENES.")

	elites = _3.collect_elite(
		sorted_population=population,
		filter_criteria=1
	)
	parents = _4.collect_parents(
		sorted_population=population,
		criteria=criteria,
		num_parents=15
	)
	family = _5.reproduce(
		parents=parents,
		mode='exponential'
	)
	shuffled_family = _1.shuffle_population(
		population=family
	)
	mutated_family = _6.mutation_round(
		shuffled_population=shuffled_family,
		partial_mutation_prob=0.000,
		pattern_mutation_prob=0.000,
		feat_idx_pool=fss	
	)
	new_population = _1.combine_populations(
		populations=[elites, mutated_family]
	)

In [ ]:
reload(_2)
_2.show_returns(population[0]._lastarray_returns, arr_close=arr_close, gene_kwargs={"population":population,"criteria":criteria,"fss":fssd})